In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn import datasets
from sklearn.metrics import confusion_matrix

In [2]:
data=pd.read_csv("bank market.csv")
data.head()

,age,job,marital,default,housing,loan,duration,campaign,y
0,56,2,1,0,0,0,261,1,no
1,39,2,1,0,1,0,935,3,yes
2,54,2,1,0,0,0,160,1,no
3,39,2,1,0,0,1,266,1,no
4,60,2,1,0,0,0,253,1,no


In [3]:
y=data["y"]
x=data.drop(columns="y")

In [4]:
y.head()

0     no
1    yes
2     no
3     no
4     no
Name: y, dtype: object

In [5]:
Y=y.values
X=x.values

In [6]:
Y

array(['no', 'yes', 'no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no',
       'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', '

In [7]:
from sklearn.metrics import roc_auc_score
from sklearn import svm

In [8]:
cv=StratifiedKFold(n_splits=2, random_state=None)
ss=svm.SVC(class_weight=None, random_state=2)
model=ss.fit(X,y)

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [9]:
parameter={
    "kernel" : ("linear","poly","rbf"),
    "C" : [0.0001,0.001,0.01,0.1,1,10,100,1000,10000],
    'gamma':[0.0001,0.001,0.01,0.1,1,10,100,1000,10000]
}
grid=GridSearchCV(model,parameter,cv=cv,n_jobs=1,scoring="accuracy")
grid

GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=None, shuffle=False),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=2,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'kernel': ('linear', 'poly', 'rbf'), 'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [10]:
from sklearn import svm
svm=svm.SVC(class_weight=None, C=1, gamma=0.1, kernel="linear", random_state=100)
n=5
from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=n, random_state=None)

In [ ]:
svm.fit(X,y)
y_pred=svm.predict(X)

In [ ]:
y_pred

In [13]:
cm=[]
total=[]
ac=[]
se=[]
sp=[]

In [14]:
for train_index, test_index in kf.split(x,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    cm.append((confusion_matrix(y_test, y_pred)).astype(float))
for j in range(n):
    total.append(sum(sum(cm[j])))
    ac.append((cm[j][0,0]+cm[j][1,1])/total[j])
    se.append(cm[j][0,0]/(cm[j][0,0]+cm[j][0,1]))
    sp.append(cm[j][1,1]/(cm[j][1,1]+cm[j][1,0]))
akurasi=np.mean(ac)
spesifisiti=np.mean(sp)
sensitiviti=np.mean(se)
print(akurasi)
print(spesifisiti)
print(sensitiviti)
#performansi masing-masing fold->csv
df_k3=pd.DataFrame()
test=dict()
for j in range(3):
    test[j]=[]
for i in range(n):
    test[0].append(ac[i])
    test[1].append(sp[i])
    test[2].append(se[i])
for i in range(3):
    df_k3=pd.concat([df_k3, pd.DataFrame(test[i])], axis=1)
df_k3.columns=['Akurasi','Spesifikasi','Sensitivitas']
df_k3

0.9652313946216384
0.4
0.9846153846153847


,Akurasi,Spesifikasi,Sensitivitas
0,0.951220,0.5,0.974359
1,0.951220,1.0,0.948718
2,0.975000,0.5,1.000000
3,0.974359,0.0,1.000000
4,0.974359,0.0,1.000000
